In [1]:
# ! pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 12.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 7.9 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 11.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 53.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 38.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.9/409.9 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 5.3 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.6/67.6 kB 12.5 MB/s eta 0:00:00
  

### Data processing from CSV to JSONL

In [1]:
import pandas as pd
import json

df = pd.read_csv("Dataset_refined.csv")

def df_to_jsonl_shuffle(df, filename):
    shuffled_df = df.sample(frac=1)  # Shuffle the DataFrame
    with open(filename, 'w') as file:
        for index, row in shuffled_df.iterrows():
            json_obj = row.to_json()
            file.write(json_obj + '\n')

def split_train_test(df, train_frac=0.9):
    train_size = int(len(df) * train_frac)
    train_df = df.iloc[:train_size]
    test_df = df.iloc[train_size:]
    return train_df, test_df

# Split DataFrame into training and test sets
train_df, test_df = split_train_test(df, train_frac=0.9)

# Convert training DataFrame to JSONL
df_to_jsonl_shuffle(train_df, 'data/train/train.jsonl')

# Convert test DataFrame to JSONL
df_to_jsonl_shuffle(test_df, 'data/validation/test.jsonl')


### prompt template creation and Prompt completion testing

In [7]:
x = {
    "java_context": "Below code demonstrates how to perform bisecting k-means clustering and evaluate the clustering performance using Spark MLlib in Java. Initially, a SparkSession is created. Then, the code loads a dataset in LIBSVM format using Spark's read method. Subsequently, a BisectingKMeans model is trained on the dataset with a specified number of clusters (K) and a seed value for reproducibility. Predictions are made on the dataset using the trained model, and the clustering quality is evaluated using the Silhouette score. Finally, the cluster centers are displayed, providing insights into the characteristics of each cluster. ",
    "java": "```java\npackage org.apache.spark.examples.ml;\n\n// $example on$\nimport org.apache.spark.ml.clustering.BisectingKMeans;\nimport org.apache.spark.ml.clustering.BisectingKMeansModel;\nimport org.apache.spark.ml.evaluation.ClusteringEvaluator;\nimport org.apache.spark.ml.linalg.Vector;\nimport org.apache.spark.sql.Dataset;\nimport org.apache.spark.sql.Row;\n// $example off$\nimport org.apache.spark.sql.SparkSession;\n\n\n/**\n * An example demonstrating bisecting k-means clustering.\n * Run with\n * <pre>\n * bin/run-example ml.JavaBisectingKMeansExample\n * </pre>\n */\npublic class JavaBisectingKMeansExample {\n\n  public static void main(String[] args) {\n    SparkSession spark = SparkSession\n      .builder()\n      .appName(\"JavaBisectingKMeansExample\")\n      .getOrCreate();\n\n    // $example on$\n    // Loads data.\n    Dataset<Row> dataset = spark.read().format(\"libsvm\").load(\"data/mllib/sample_kmeans_data.txt\");\n\n    // Trains a bisecting k-means model.\n    BisectingKMeans bkm = new BisectingKMeans().setK(2).setSeed(1);\n    BisectingKMeansModel model = bkm.fit(dataset);\n\n    // Make predictions\n    Dataset<Row> predictions = model.transform(dataset);\n\n    // Evaluate clustering by computing Silhouette score\n    ClusteringEvaluator evaluator = new ClusteringEvaluator();\n\n    double silhouette = evaluator.evaluate(predictions);\n    System.out.println(\"Silhouette with squared euclidean distance = \" + silhouette);\n\n    // Shows the result.\n    System.out.println(\"Cluster Centers: \");\n    Vector[] centers = model.clusterCenters();\n    for (Vector center : centers) {\n      System.out.println(center);\n    }\n    // $example off$\n\n    spark.stop();\n  }\n}```",
    "python_context": "Python equivalent code which demonstrates how to perform bisecting k-means clustering and evaluate the clustering performance utilizes PySpark's MLlib library. Firstly, a SparkSession is initialized. Then, the code loads a dataset in LIBSVM format using Spark's read method. Subsequently, a BisectingKMeans model is trained on the dataset with a specified number of clusters (K) and a seed value for reproducibility. Predictions are made on the dataset using the trained model, and the clustering quality is evaluated using the Silhouette score. Finally, the cluster centers are displayed, offering insights into the characteristics of each cluster. ",
    "python": "```python\nfrom pyspark.ml.clustering import BisectingKMeans\nfrom pyspark.ml.evaluation import ClusteringEvaluator\n# $example off$\nfrom pyspark.sql import SparkSession\n\nif __name__ == \"__main__\":\n    spark = SparkSession\\\n        .builder\\\n        .appName(\"BisectingKMeansExample\")\\\n        .getOrCreate()\n\n    # $example on$\n    # Loads data.\n    dataset = spark.read.format(\"libsvm\").load(\"data/mllib/sample_kmeans_data.txt\")\n\n    # Trains a bisecting k-means model.\n    bkm = BisectingKMeans().setK(2).setSeed(1)\n    model = bkm.fit(dataset)\n\n    # Make predictions\n    predictions = model.transform(dataset)\n\n    # Evaluate clustering by computing Silhouette score\n    evaluator = ClusteringEvaluator()\n\n    silhouette = evaluator.evaluate(predictions)\n    print(\"Silhouette with squared euclidean distance = \" + str(silhouette))\n\n    # Shows the result.\n    print(\"Cluster Centers: \")\n    centers = model.clusterCenters()\n    for center in centers:\n        print(center)\n    # $example off$\n\n    spark.stop()```"
}

In [2]:
import json

template = {
    "prompt": """Below is an instruction that describes a problem and its code implementation in Java. Write a response which converts the Java implementation to an implementation in Python.
Problem Context:
{java_context}
Code in Java:
{java}
""",
    "completion": """ 
Solution Context:
{python_context}

Code in Python:
{python}
""",
}
with open("./data/template.json", "w") as f:
    json.dump(template, f)

In [7]:
import json

# Open the JSON file
with open('./data/template.json', 'r') as file:
    # Load the JSON data
    data = json.load(file)

# Now you can access the data as a dictionary
# For example, if your JSON file has a key named 'name', you can access it like this:
name = data
name

{'prompt': 'Below is an instruction that describes a problem and its code implementation in Java. Write a response which converts the Java implementation to an implementation in Python.\n\nProblem Context:\n{java_context}\n\nCode in Java:\n{java}\n',
 'completion': ' \nSolution Context:\n{python_context}\n                        \nCode in Python:\n{python}\n'}

In [9]:
prompt_template = f"""### Instruction:

{template['prompt']}

### Response:

{template['completion']}

"""

prompt_template

'### Instruction:\n\nBelow is an instruction that describes a problem and its code implementation in Java. Write a response which converts the Java implementation to an implementation in Python.\n\nProblem Context:\n{java_context}\n\nCode in Java:\n{java}\n\n\n### Response:\n\n \nSolution Context:\n{python_context}\n                        \nCode in Python:\n{python}\n\n\n'

In [10]:
y = prompt_template.format(**x)
y

'### Instruction:\n\nBelow is an instruction that describes a problem and its code implementation in Java. Write a response which converts the Java implementation to an implementation in Python.\n\nProblem Context:\nBelow code demonstrates how to perform bisecting k-means clustering and evaluate the clustering performance using Spark MLlib in Java. Initially, a SparkSession is created. Then, the code loads a dataset in LIBSVM format using Spark\'s read method. Subsequently, a BisectingKMeans model is trained on the dataset with a specified number of clusters (K) and a seed value for reproducibility. Predictions are made on the dataset using the trained model, and the clustering quality is evaluated using the Silhouette score. Finally, the cluster centers are displayed, providing insights into the characteristics of each cluster. \n\nCode in Java:\n```java\npackage org.apache.spark.examples.ml;\n\n// $example on$\nimport org.apache.spark.ml.clustering.BisectingKMeans;\nimport org.apache

Note to self: 
- Loading a 13B Code Llama model requires instances bigger than g5.12xlarge and g4dn.12xlarge
- Both have 4 GPU's 96GB GPU memory, 48vCPU's and 192GB CPU memory (RAM)
- When loading the 13B pretrained model, the RAM is not sufficient. Need a bigger instance
- If I force the model to load using GPU, by setting device_map="cuda" in AutoModelForCausalLM.from_pretrained, I'm hitting GPU OOM errors
    -  CUDA out of memory. Tried to allocate 100.00 MiB. GPU 0 has a total capacity of 21.99 GiB of which 47.06 MiB is free. 
- Using a 13B model for testing seems to be greedy. Switching to 7B models

In [ ]:
%%sh

python train.py \
    --model_dir ./models/CodeLlama-7b-Python-HF \
    --enable_fsdp True \
    --fsdp_checkpoint_root_dir ./checkpoints/CodeLlama-7b-Python-HF \
    --num_epochs 1 \
    --int8_quantization False \
    --learning_rate 0.001 \
    --seed 10 \
    --use_peft True \
    --peft_output_dir ./output/CodeLlama-7b-Python-HF \
    --train_dir ./data/train \
    --validation_dir ./data/validation \
    --file_extension jsonl \
    --prompt_template ./data/template.json
    --model_output_dir ./finetuned_model/CodeLlama-7b-Python-HF/run1

INFO:root:Finetuning Args: Namespace(model_dir='./models/CodeLlama-7b-Python-HF', per_device_train_batch_size=4, batching_strategy='packing', context_length=4096, gradient_accumulation_steps=1, gradient_clipping=False, gradient_clipping_threshold=1.0, num_epochs=1, num_workers_dataloader=1, learning_rate=0.001, weight_decay=0.0, gamma=0.85, seed=10, int8_quantization=False, freeze_layers=False, num_freeze_layers=1, use_fast_kernels=False, save_metrics=False, run_validation=True, val_batch_size=1, enable_fsdp=True, fsdp_checkpoint_root_dir='./checkpoints/CodeLlama-7b-Python-HF', low_cpu_fsdp=False, mixed_precision=True, use_fp16=False, pure_bf16=False, optimizer='AdamW', save_optimizer=False, use_peft=True, peft_method='lora', peft_output_dir='./output/CodeLlama-7b-Python-HF', lora_r=8, lora_alpha=32, lora_dropout=0.05, target_modules='q_proj,v_proj', train_dir='./data/vivek/train', validation_dir='./data/vivek/validation', file_extension='jsonl', prompt_template='./data/vivek/template.

--> Running with torch dist debug set to detail


[W Utils.hpp:133] Warning: Environment variable NCCL_ASYNC_ERROR_HANDLING is deprecated; use TORCH_NCCL_ASYNC_ERROR_HANDLING instead (function getCvarInt)
[W Utils.hpp:133] Warning: Environment variable NCCL_ASYNC_ERROR_HANDLING is deprecated; use TORCH_NCCL_ASYNC_ERROR_HANDLING instead (function getCvarInt)
INFO:root:Local rank is 2. Rank is 2. World Size is 4
INFO:root:Setting torch device = 2
INFO:root:Local rank is 3. Rank is 3. World Size is 4
INFO:root:Setting torch device = 3
[W Utils.hpp:133] Warning: Environment variable NCCL_ASYNC_ERROR_HANDLING is deprecated; use TORCH_NCCL_ASYNC_ERROR_HANDLING instead (function getCvarInt)
INFO:root:Local rank is 1. Rank is 1. World Size is 4
INFO:root:Setting torch device = 1
You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers
INFO:root:Loading the tokenizer.
INFO:root:Loading the tokenizer.
INFO:root:Loading the tokenizer.
You set `add_prefix_space`. The tokenizer needs to be converted from the slow to

--> Model ./models/CodeLlama-7b-Python-HF

--> ./models/CodeLlama-7b-Python-HF has 6738.415616 Million params



INFO:root:Using PEFT


...

evaluating Epoch:   0%|          | 0/1 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_

 eval_ppl=tensor(1.9563, device='cuda:0') eval_epoch_loss=tensor(0.6711, device='cuda:0')
we are about to save the PEFT modules


/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/peft/utils/save_and_load.py:154: UserWarning: Could not find a config file in ./models/CodeLlama-7b-Python-HF - will assume that the vocabulary was not modified.
  warnings.warn(
/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/peft/utils/save_and_load.py:154: UserWarning: Could not find a config file in ./models/CodeLlama-7b-Python-HF - will assume that the vocabulary was not modified.
  warnings.warn(
/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/peft/utils/save_and_load.py:154: UserWarning: Could not find a config file in ./models/CodeLlama-7b-Python-HF - will assume that the vocabulary was not modified.
  warnings.warn(
/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/peft/utils/save_and_load.py:154: UserWarning: Could not find a config file in ./models/CodeLlama-7b-Python-HF - will assume that the vocabulary was not modified.
  warnings.wa

PEFT modules are saved in ./output/CodeLlama-7b-Python-HF directory
best eval loss on epoch 1 is 0.671068012714386
Epoch 1: train_perplexity=2.0012, train_epoch_loss=0.6938, epoch time 56.48963721600012s


INFO:root:Training process complete
INFO:root:Training process complete
INFO:root:Training process complete
INFO:root:Training process complete
INFO:root:Key: avg_train_prep, Value: 2.001217842102051
INFO:root:Key: avg_train_loss, Value: 0.6937559843063354
INFO:root:Key: avg_eval_prep, Value: 1.9563255310058594
INFO:root:Key: avg_eval_loss, Value: 0.671068012714386
INFO:root:Key: avg_epoch_time, Value: 56.48963721600012
INFO:root:Key: avg_checkpoint_time, Value: 9.73087351400045
INFO:root:Combining pre-trained base model with the PEFT adapter module.
Loading checkpoint shards: 100%|██████████| 3/3 [00:01<00:00,  2.93it/s]
INFO:root:Saving the combined model in safetensors format.
INFO:root:Saving complete.
INFO:root:Saving the tokenizer.
You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers
INFO:root:Saving complete.


In [5]:
# Retrying the above command to save peft model as I made a mistake of passing incorrect arg parameter for base model

### Training CodeLlama Python-7b Run 1

In [3]:
%%sh

python train.py \
    --model_dir ./models/CodeLlama-7b-Python-HF \
    --enable_fsdp True \
    --fsdp_checkpoint_root_dir ./checkpoints/CodeLlama-7b-Python-HF \
    --num_epochs 1 \
    --int8_quantization False \
    --learning_rate 0.001 \
    --seed 10 \
    --use_peft True \
    --peft_output_dir ./output/CodeLlama-7b-Python-HF \
    --train_dir ./data/train \
    --validation_dir ./data/validation \
    --file_extension jsonl \
    --prompt_template ./data/template.json \
    --model_output_dir ./finetuned_model/CodeLlama-7b-Python-HF/run1

INFO:root:Finetuning Args: Namespace(model_dir='./models/CodeLlama-7b-Instruct-HF', per_device_train_batch_size=4, batching_strategy='packing', context_length=4096, gradient_accumulation_steps=1, gradient_clipping=False, gradient_clipping_threshold=1.0, num_epochs=5, num_workers_dataloader=1, learning_rate=0.001, weight_decay=0.0, gamma=0.85, seed=10, int8_quantization=False, freeze_layers=False, num_freeze_layers=1, use_fast_kernels=False, save_metrics=False, run_validation=True, val_batch_size=1, enable_fsdp=True, fsdp_checkpoint_root_dir='./checkpoints/CodeLlama-7b-Instruct-HF', low_cpu_fsdp=False, mixed_precision=True, use_fp16=False, pure_bf16=False, optimizer='AdamW', save_optimizer=False, use_peft=True, peft_method='lora', peft_output_dir='./output/CodeLlama-7b-Instruct-HF', lora_r=8, lora_alpha=32, lora_dropout=0.05, target_modules='q_proj,v_proj', train_dir='./data/vivek/train', validation_dir='./data/vivek/validation', file_extension='jsonl', prompt_template='./data/vivek/tem

--> Running with torch dist debug set to detail


You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers
INFO:root:Loading the tokenizer.
You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers
INFO:root:Loading the tokenizer.
You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers
INFO:root:Loading the tokenizer.
You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers
INFO:root:Using the default value of max_input_length=2048.
INFO:root:--> Training Set Length = 107
INFO:root:--> Validation Set Length = 7
INFO:root:Loading the pre-trained model and setup its configuration
INFO:root:Model Name: ./models/CodeLlama-7b-Instruct-HF
INFO:root:enable_fsdp is set to True and low_cpu_fsdp is set to False
INFO:root:Using the default value of max_input_length=2048.
INFO:root:Using the default value of max_input_length=2048.
INFO:root:Loading the pre-trained model and setup its configuration
INFO:root:Model Name: .

--> Model ./models/CodeLlama-7b-Instruct-HF

--> ./models/CodeLlama-7b-Instruct-HF has 6738.546688 Million params



INFO:root:Using PEFT


...

Training Epoch: 5/5, step 5/6 completed (loss: 0.2932030260562897): 100%|██████████| 6/6 [01:18<00:00, 13.06s/it]]
Training Epoch: 5/5, step 5/6 completed (loss: 0.2609100043773651): 100%|██████████| 6/6 [01:18<00:00, 13.06s/it]
Training Epoch: 5/5, step 5/6 completed (loss: 0.3223790228366852): 100%|██████████| 6/6 [01:18<00:00, 13.06s/it]
Training Epoch: 5/5, step 5/6 completed (loss: 0.27691683173179626): 100%|██████████| 6/6 [01:18<00:00, 13.06s/it]


Max CUDA memory allocated was 18 GB
Max CUDA memory reserved was 20 GB
Peak active CUDA memory was 18 GB
CUDA Malloc retries : 401
CPU Total Peak Memory consumed during the train (max): 2 GB


evaluating Epoch: 100%|██████████| 2/2 [00:05<00:00,  2.87s/it]

evaluating Epoch: 100%|██████████| 2/2 [00:05<00:00,  2.87s/it]


 eval_ppl=tensor(1.7366, device='cuda:0') eval_epoch_loss=tensor(0.5520, device='cuda:0')
Epoch 5: train_perplexity=1.4203, train_epoch_loss=0.3508, epoch time 78.84463565199985s


INFO:root:Training process complete
INFO:root:Training process complete
INFO:root:Training process complete
INFO:root:Training process complete
INFO:root:Key: avg_train_prep, Value: 1.5957965135574341
INFO:root:Key: avg_train_loss, Value: 0.46181475520133974
INFO:root:Key: avg_eval_prep, Value: 1.7371895790100098
INFO:root:Key: avg_eval_loss, Value: 0.5498505353927612
INFO:root:Key: avg_epoch_time, Value: 79.32849227060069
INFO:root:Key: avg_checkpoint_time, Value: 5.268354036599339
INFO:root:Combining pre-trained base model with the PEFT adapter module.
Loading checkpoint shards: 100%|██████████| 3/3 [00:00<00:00,  3.00it/s]
INFO:root:Saving the combined model in safetensors format.
INFO:root:Saving complete.
INFO:root:Saving the tokenizer.
You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers
INFO:root:Saving complete.
INFO:root:Model Tarring process has begun
INFO:root:Uploading to S3
INFO:botocore.credentials:Found credentials from IAM Role: BaseN

In [ ]:
nohup python train.py --model_dir ./models/CodeLlama-7b-Python-HF --enable_fsdp True --fsdp_checkpoint_root_dir ./checkpoints/CodeLlama-7b-Python-HF --num_epochs 5 \--int8_quantization False \
    --learning_rate 0.001 \
    --seed 10 \
    --use_peft True \
    --peft_output_dir ./output/CodeLlama-7b-Python-HF \
    --train_dir ./data/train \
    --validation_dir ./data/validation \
    --file_extension jsonl \
    --prompt_template ./data/template.json \
    --model_output_dir ./finetuned_model/CodeLlama-7b-Python-HF/run1 &